In [1]:
from data_preparation import load_data_tensor, split_train_data

In [2]:
import torch
import numpy as np
import torch.optim as optim
from sklearn.model_selection import KFold
from GSR_Net.preprocessing import *
from GSR_Net.model import *
from GSR_Net.train import *
import argparse

seed_value = 42
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [3]:
# Check for CUDA availability
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print(f"CUDA available... Using {DEVICE}!")
else:
    DEVICE = torch.device('cpu')
    print(f"CUDA not available... Using {DEVICE}!")

CUDA available... Using cuda!


In [4]:
path_to_data = '/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/data'

x_train, x_test, y_train = load_data_tensor(path_to_data)

In [5]:
X, Y = x_train.to(DEVICE), y_train.to(DEVICE)

In [6]:
class ModelArgs:  
    def __init__(self, epochs, lr, splits, lmbda, lr_dim, hr_dim, hidden_dim,
                 padding):
        self.epochs = epochs
        self.lr = lr
        self.splits = splits
        self.lmbda = lmbda
        self.lr_dim = lr_dim
        self.hr_dim = hr_dim
        self.hidden_dim = hidden_dim
        self.padding = padding

args = ModelArgs(
    epochs=200,
    lr=0.0001,
    splits=5,
    lmbda=16,
    lr_dim=160,
    hr_dim=320,
    hidden_dim=320,
    padding=26,
)

In [7]:
cv = KFold(n_splits=args.splits, random_state=42, shuffle=True)
ks = [0.9, 0.7, 0.6, 0.5]


In [8]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args).to(DEVICE)
print(model)

GSRNet(
  (layer): GSRLayer()
  (net): GraphUnet(
    (start_gcn): GCN(
      (proj): Linear(in_features=160, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (bottom_gcn): GCN(
      (proj): Linear(in_features=320, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (end_gcn): GCN(
      (proj): Linear(in_features=640, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
  )
  (gc1): GraphConvolution()
  (gc2): GraphConvolution()
)


/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/layers.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.weights = torch.nn.Parameter(data=torch.tensor(initial_weights, device=self.device, dtype=torch.float), requires_grad=True)


In [9]:
def train_model():
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    for train_index, test_index in cv.split(X):
        subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = X[
            train_index], X[test_index], Y[train_index], Y[test_index]
        train(model, optimizer, subjects_adj, subjects_ground_truth, args)
        test(model, test_adj, test_ground_truth, args)
    

In [10]:
train_model()

/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/preprocessing.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float32)


Epoch:  1 Loss:  0.8746659424071922 Error:  25.988830446748807 %
Epoch:  2 Loss:  0.5940021142027432 Error:  25.91338502733331 %
Epoch:  3 Loss:  0.45791336602734445 Error:  25.802313004221233 %
Epoch:  4 Loss:  0.3801361371700029 Error:  25.585741936264185 %
Epoch:  5 Loss:  0.33066079580694213 Error:  24.85881592322113 %
Epoch:  6 Loss:  0.297476619706118 Error:  23.79708613892247 %
Epoch:  7 Loss:  0.28054673443163247 Error:  23.24816090495963 %
Epoch:  8 Loss:  0.27178493411021126 Error:  22.929271092092186 %
Epoch:  9 Loss:  0.2674908011703563 Error:  22.7088814615307 %
Epoch:  10 Loss:  0.2655556626561889 Error:  22.53211713150928 %
Epoch:  11 Loss:  0.264357515519723 Error:  22.397799957963755 %
Epoch:  12 Loss:  0.2632356515728441 Error:  22.28932780654807 %
Epoch:  13 Loss:  0.26203043243490665 Error:  22.203688973322848 %
Epoch:  14 Loss:  0.2612084239945376 Error:  22.132100276929094 %
Epoch:  15 Loss:  0.26038244411461337 Error:  22.069637466194038 %
Epoch:  16 Loss:  0.259